<a href="https://colab.research.google.com/github/hanz13/machine-learning-dicoding/blob/main/proyek_submission_dicoding_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perkenalan

Nama: Subhan Ashof

Proyek Submission: Membuat klasifikasi gambar batu, gunting , kentas dengan Alogoritma menggunakan Tensorflow.




# Import Libraries

In [ ]:
import os
import glob
import random
import shutil
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input


import warnings
warnings.filterwarnings('ignore')

# Data Loading

In [ ]:
#download dataset
!wget --no-check-certificate \
https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

In [ ]:
folder_to_delete = '/content/dataset/'

try:
    shutil.rmtree(folder_to_delete)
    print(f'Folder {folder_to_delete} dan isinya berhasil dihapus.')
except Exception as e:
    print(f'Gagal menghapus folder: {str(e)}')

In [ ]:
#unzip rockpaperscissors.zip dan simpan ke folder dataset
!unzip -q rockpaperscissors.zip -d dataset

In [ ]:
#mengecek isi dir di dalam folder main_path
main_path= '/content/dataset/rockpaperscissors'
os.listdir(main_path)

In [ ]:
def check_dir(dir: str = None, labels: list = None):
    '''
    Mengecek jumlah data dalam direktori berdasarkan label.

    Parameters:
        dir (str): Path ke direktori yang akan diperiksa.
        labels (list): List dari label yang akan dicari dalam direktori.

    Returns:
        None

    Contoh Penggunaan:
    ```python
    train_dir = "/path/to/train_directory"
    test_dir = "/path/to/test_directory"

    print('Train set: \n' + '='*50)
    check_dir(train_dir, ['PNEUMONIA', 'NORMAL'])

    print('\nTest set: \n' + '='*50)
    check_dir(test_dir, ['PNEUMONIA', 'NORMAL'])
    ```

    Mencetak jumlah data untuk setiap label yang ditemukan dalam direktori.
    '''
    for label in labels:
        num_data = len(os.listdir(os.path.join(dir, label)))
        print(f'Jumlah {label}: {num_data}')

In [ ]:
print('Jumlah gambar pada setiap label: \n' + '='*50)
check_dir(main_path,['rock','paper','scissors'])

In [ ]:
def create_directories(base_dir=None, sub_dir=None, labels=None):
    '''
    Membuat struktur direktori untuk dataset berdasarkan label.

    Parameters:
        base_dir (str): Path ke direktori utama.
        labels (list): List dari label yang akan digunakan.
        sub_dir (list): List dari nama direktori yang akan dibuat.

    Returns:
        None
    '''
    # Buat direktori 'train' dan/atau 'test'
    # Jika direktori tersebut sudah ada maka akan melanjutkan dan tidak akan muncul error.
    for sub in sub_dir:
        dir_path = os.path.join(base_dir, sub)
        os.makedirs(dir_path, exist_ok=True)

    # Buat direktori berisi nama labels
    for label in labels:
        for sub in sub_dir:
            label_dir = os.path.join(base_dir, sub, label)
            os.makedirs(label_dir, exist_ok=True)

    return f'Telah berhasil membuat sub directories :{sub_dir} dan labels: {labels}'

In [ ]:
labels = ['rock', 'paper', 'scissors']
list_sub_dir = ['train', 'val']

#panggil prosedur create_directories
create_directories(main_path,  list_sub_dir, labels)

In [ ]:
# Path ke folder 'train', dan 'test'
train_folder = os.path.join(main_path,'train')
val_folder = os.path.join(main_path,'val')

# path ke folder 'rock', 'paper', 'scissors' (tempat file gambar)
rock_folder = os.path.join(main_path,'rock')
paper_folder = os.path.join(main_path,'paper')
scissors_folder = os.path.join(main_path,'scissors')


In [ ]:
train_folder

Selanjutnya saya akan membuat sendiri perintah untuk splitting data tanpa menggunakan scikit learn.

In [ ]:
def split_data(source=None, destination=None, label=None,split_percentage=None):
    '''
    Memindahkan data dan Memisahkan data dari sumber ke destinasi dengan persentase tertentu.

    Parameters:
        source (str): Path direktori sumber data yang akan dipindahkan.
        destination (str): Path ke direktori tujuan.
        label (list): Nama label yang akan digunakan.
        split_percentage (float): Persentase data yang akan dipindahkan ke destinasi.

    Returns:
        str: Pesan sukses ketika operasi selesai.

    Examples:
    ```python
    source_folder = 'source'
    destination_folder = 'destination'
    split_percentage = 0.6
    split_data(source_folder, destination_folder, split_percentage)
    ```

    Fungsi ini memindahkan data dari direktori sumber ke destinasi dengan persentase tertentu,
    seperti membagi data menjadi train dan test set.
    '''
    files = os.listdir(source)
    num_files = len(files)
    num_train = int(num_files * split_percentage)

    # Acak urutan file
    random.shuffle(files)

    train_files = files[:num_train]
    test_files = files[num_train:]

    # Pindahkan file ke folder tujuan
    for file in train_files:
        source_file = os.path.join(source, file)
        destination_file = os.path.join(destination[0],label)
        shutil.copy(source_file, destination_file)

    for file in test_files:
        source_file = os.path.join(source, file)
        destination_file = os.path.join(destination[1],label)
        shutil.copy(source_file, destination_file)

    return 'Telah berhasil memasukkan data'


In [ ]:
# Persentase data yang akan digunakan sebagai split data
train_percentage = 0.6

#list folder train dan test
destination_folder= [train_folder,val_folder]
# Bagi data untuk masing-masing label
split_data(rock_folder, destination_folder, 'rock', train_percentage)
split_data(paper_folder, destination_folder, 'paper', train_percentage)
split_data(scissors_folder, destination_folder,'scissors', train_percentage)

In [ ]:
print('Jumlah gambar di train: \n' + '='*50)
check_dir(train_folder,['rock','paper','scissors'])

print('Jumlah gambar di test: \n' + '='*50)
check_dir(val_folder,['rock','paper','scissors'])

In [ ]:
def visualize_random_data(data_dir, label, num_samples=5):
    label_dir = os.path.join(data_dir, label)
    image_files = os.listdir(label_dir)

    num_samples = min(num_samples, len(image_files))
    random_images = random.sample(image_files, num_samples)

    fig, axes = plt.subplots(1, num_samples, figsize=(10, 2), constrained_layout=True)

    # Kurangi jarak antara subplot horizontal dan vertikal
    plt.subplots_adjust(wspace=0.2, hspace=0.2)

    for j, random_image in enumerate(random_images):
        image_path = os.path.join(label_dir, random_image)

        img = mpimg.imread(image_path)
        image_size = f'Size: {img.shape[1]} x {img.shape[0]}'  # Menampilkan ukuran gambar
        title = f'Label: {label}\n{image_size}'  # Gabungkan label dan ukuran

        axes[j].imshow(img)
        axes[j].set_title(title)
        axes[j].axis('off')

    plt.show()


label_to_visualize = 'rock'
visualize_random_data(main_path, label=label_to_visualize, num_samples=5)

In [ ]:
visualize_random_data(main_path, label='paper', num_samples=5)

In [ ]:
visualize_random_data(main_path, label='scissors', num_samples=5)

# Data preprocessing

In [ ]:
img_height= 150
img_width= 150
input_size= (img_height,img_width)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 90,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    brightness_range=[0.5, 1.5],
    width_shift_range=0.1,
    height_shift_range=0.1,
)
val_datagen = ImageDataGenerator(
    rescale = 1./225,
    rotation_range = 90,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    brightness_range=[0.5, 1.5],
    width_shift_range=0.1,
    height_shift_range=0.1,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=input_size,
    batch_size= 32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_folder,
    target_size = input_size,
    batch_size = 32,
    class_mode = 'categorical',
    shuffle=False
)

In [ ]:
class_indices = train_generator.class_indices
print(class_indices)

In [ ]:
def display_augmented_images(data_generator, num_samples_to_display=10):
    num_rows= 2
    num_cols = int(num_samples_to_display/num_rows)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6))
    for i in range(num_rows):
        for j in range(num_cols):
            batch = data_generator.next()
            image = batch[0][0]  # Ambil gambar pertama dari batch
            label = batch[1][0]  # Ambil label pertama dari batch

            # Konversi label dalam format one-hot encoding ke kelas asli
            class_index = label.argmax()

            # Menampilkan gambar
            axes[i, j].imshow(image)
            axes[i, j].set_title(f"Class: {class_index}")
            axes[i, j].axis('off')

    plt.show()

display_augmented_images(train_generator, num_samples_to_display=10)


# Sequatial Model

In [ ]:
input_shape=(img_height,img_width,3)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape= input_shape),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation= 'relu'),
  tf.keras.layers.Dense(128, activation= 'relu'),
  tf.keras.layers.Dense(3, activation= 'softmax')
])

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.experimental.Nadam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience= 3, min_delta=0.1),
    ModelCheckpoint('model_best2.h5', monitor='val_loss', save_best_only=True)
]

Pada code dibawah ini validation_Steps dan steps_per_epoch secara otomatis akan sesuai dengan jumlah batch dan images. Rumusnya `number_images/batch_size`

In [ ]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    # validation_steps = 14, #  number_images/batch_size = 876/64
    # steps_per_epoch = 21, #  number_images/batch_size= 1312/64
    epochs = 100,
    callbacks= callbacks
)

# Evaluasi Model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# Membuat dataframe
data = {
    'Epoch': range(1, len(acc) + 1),
    'Accuracy': acc,
    'Validation Accuracy': val_acc,
    'Loss': loss,
    'Validation Loss': val_loss
}

df = pd.DataFrame(data)

# Menampilkan dataframe
display(df)

In [ ]:
epochs_range= range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# load model_best2.h5
# Ganti dengan path ke file model_best2.h5
model_path = '/content/model_best2.h5'

# Memuat model
loaded_model = load_model(model_path)

In [ ]:
#evaluasi dengan model_best2.h5
test_results_best = loaded_model.evaluate(validation_generator, verbose=0)
print(f'Test Loss     : {test_results_best[0]:.4f}')
print(f'Test Accuracy : {test_results_best[1]:.4f}')

In [ ]:
#evaluasi model bukan dari model_best2.h5
test_results = model.evaluate(validation_generator, verbose=0)
print(f'Test Loss     : {test_results[0]:.4f}')
print(f'Test Accuracy : {test_results[1]:.4f}')

Dapat kita lihat bahwa dengan menggunakan modelcheckpoint `model_best2.h5` , kita dapat hasil yang lebih bagus, karena sebelumnya kita telah atur ke `save_best_only=True`

In [ ]:
class_indices = train_generator.class_indices
class_indices.keys()

In [ ]:
# Fungsi untuk mengunggah dan memprediksi gambar
def upload_and_predict_image(model):
    # Upload gambar
    uploaded = files.upload()

    # Mendapatkan label hasil prediksi
    labels = ['paper', 'rock', 'scissors']

    for image_path in uploaded.keys():
        # Augmentasi gambar dengan random_invert
        img = image.load_img(image_path, target_size=(150, 150))
        img = image.img_to_array(img)
        img = img / 255.0  # Preprocessing gambar (normalisasi)


        img = np.expand_dims(img, axis=0)

        # Melakukan prediksi
        prediction = loaded_model.predict(img)
        print(prediction)

        # Mendapatkan hasil prediksi
        predicted_class = labels[np.argmax(prediction)]
        predicted_probability = np.max(prediction)

        # Menampilkan gambar
        plt.figure()
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.title(f'Hasil prediksi: {predicted_class}\nProbabilitas: {predicted_probability:.2f}')
        plt.axis('off')
        plt.show()

# Contoh penggunaan
# Gantilah loaded_model dengan model yang sesuai dengan tugas Anda
upload_and_predict_image(loaded_model)
